In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime
import numpy as np

In [ ]:
data = pd.read_csv(r"C:\Geospatial Data Science_GEOG490\Madis_Locations.csv")

# converting 'Date' and 'Arrival Time' to datetime
data['Date'] = pd.to_datetime(data['Date'])
data['Arrival Time'] = pd.to_datetime(data['Arrival Time'])

# extracting features from 'Date' and 'Arrival Time'
data['DOW'] = data['Date'].dt.dayofweek
data['Hour'] = data['Arrival Time'].dt.hour

# defining the features and the target
X = data[['DOW', 'Hour']]
y_lat = data['lat']
y_lon = data['lon']
addresses = data[['lat', 'lon', 'Location']]

# splitting the data into training and testing sets
X_train, X_test, y_lat_train, y_lat_test, y_lon_train, y_lon_test = train_test_split(X, y_lat, y_lon, test_size=0.2, random_state=42)

# training the models
lat_model = RandomForestRegressor()
lon_model = RandomForestRegressor()

lat_model.fit(X_train, y_lat_train)
lon_model.fit(X_train, y_lon_train)

# predicting on the test set
y_lat_pred = lat_model.predict(X_test)
y_lon_pred = lon_model.predict(X_test)

# evaluating the models
lat_mse = mean_squared_error(y_lat_test, y_lat_pred)
lon_mse = mean_squared_error(y_lon_test, y_lon_pred)

print(f'Latitude MSE: {lat_mse}')
print(f'Longitude MSE: {lon_mse}')


def find_nearest_address(lat, lon, addresses):
    distances = np.sqrt((addresses['lat'] - lat)**2 + (addresses['lon'] - lon)**2)
    nearest_index = distances.idxmin()
    return addresses.loc[nearest_index, 'Location']


def predict_location(dow, hour):
    input_data = pd.DataFrame([[dow, hour]], columns=['DOW', 'Hour'])
    lat_pred = lat_model.predict(input_data)[0]
    lon_pred = lon_model.predict(input_data)[0]
    nearest_address = find_nearest_address(lat_pred, lon_pred, addresses)
    return lat_pred, lon_pred, nearest_address

'''Enter in day of the week and hour you would like to find Madi
   For the day of the week enter in an integer from the following:
   0 = Monday 
   1 = Tuesday
   2 = Wednesday
   3 = Thursday
   4 = Friday
   5 = Satuday
   6 = Sunday 
   
   For the hour of the day enter in 1-24'''
dow = 3
hour = 10
predicted_lat, predicted_lon, predicted_address = predict_location(dow, hour)
print('On day {}, Madi should be at ({}, {}), {}, at {}'.format(dow, predicted_lat, predicted_lon, predicted_address, hour))
